https://discourse.julialang.org/t/how-to-correctly-define-and-use-global-variables-in-the-module-in-julia/65720/8

In [1]:
# The `::Float64`'s below are redundant.
module A
global a = 100.0::Float64
function f(x)::Float64
    a = x^2
    return a
end
println("f(6.0) = ", f(6.0), ",  a = ", a)
end;

f(6.0) = 36.0,  a = 100.0


In [2]:
# A is equivalent to the following.
module B
a = 100.0
function f(x)::Float64
    a = x^2
    return a
end
println("f(6.0) = ", f(6.0), ",  a = ", a)
end;

f(6.0) = 36.0,  a = 100.0


In [3]:
# Don't write return-type in general.
# https://github.com/JuliaLang/julia/blob/master/doc/src/manual/functions.md#return-type
module B′
a = 100.0
function f(x)
    a = x^2
    return a
end
println("f(6.0) = ", f(6.0), ",  a = ", a)
end;

f(6.0) = 36.0,  a = 100.0


In [4]:
# We need `global` for the immutable global variable `a` in for-loop.
module C
a = 100.0
function f(x)
    global a = x^2
    return a
end
println("f(6.0) = ", f(6.0), ",  a = ", a)
end;

f(6.0) = 36.0,  a = 36.0


In [5]:
# We don't need `global` for the content `a[]` of the mutable global variable `a`.
module D
a = Ref(100.0)
function f(x)
    a[] = x^2
    return a[]
end
println("f(6.0) = ", f(6.0), ",  a[] = ", a[])
end;

f(6.0) = 36.0,  a[] = 36.0


In [6]:
# D is type-unstable.  We need `const` for type stability.
module E
const a = Ref(100.0)
function f(x)
    a[] = x^2
    return a[]
end
println("f(6.0) = ", f(6.0), ",  a[] = ", a[])
end;

f(6.0) = 36.0,  a[] = 36.0


In [7]:
@code_warntype D.f(6.0)

MethodInstance for Main.D.f(::Float64)
  from f(x) in Main.D at In[5]:4
Arguments
  #self#::Core.Const(Main.D.f)
  x::Float64
Body::Any
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %2 = (%1)()::Core.Const(Val{2}())
│   %3 = Base.literal_pow(Main.D.:^, x, %2)::Float64
│        Base.setindex!(Main.D.a, %3)
│   %5 = Base.getindex(Main.D.a)::Any
└──      return %5



In [8]:
@code_warntype E.f(6.0)

MethodInstance for Main.E.f(::Float64)
  from f(x) in Main.E at In[6]:4
Arguments
  #self#::Core.Const(Main.E.f)
  x::Float64
Body::Float64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %2 = (%1)()::Core.Const(Val{2}())
│   %3 = Base.literal_pow(Main.E.:^, x, %2)::Float64
│        Base.setindex!(Main.E.a, %3)
│   %5 = Base.getindex(Main.E.a)::Float64
└──      return %5

